In [39]:
from numba import njit, jit

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from numpy import random
from numpy import linalg
from scipy import sparse
from scipy import stats
from scipy.optimize import minimize
import statsmodels.api as sm
import matplotlib
import itertools as it
from matplotlib import rc


import pandas as pd
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = [
    r'\usepackage{amssymb}',
    r'\usepackage{amsmath}',
    r'\usepackage{xcolor}',
    r'\renewcommand*\familydefault{\sfdefault}']
matplotlib.rcParams['pgf.texsystem'] = 'pdflatex'
matplotlib.rcParams['pgf.preamble']  = [
    r'\usepackage[utf8x]{inputenc}',
    r'\usepackage{amssymb}',
    r'\usepackage[T1]{fontenc}',
    r'\usepackage{amsmath}',
    r'\usepackage{sansmath}']

from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

import warnings

warnings.filterwarnings('ignore')

In [32]:
def lasso_objective(b,data,lmbda):
    p = b.size
    y = data[:,0]
    X = data[:,np.arange(1,data[0,:].size)]
    N = y.size
    pdata = X[0,:].size
    if p!=pdata:
        print("Error: Covariate number is not compatible with given coefficient vector")
        return
    else:
        obj = np.square(y-X@b).sum() + linalg.norm(b,ord=1)
        return obj
        


  



In [107]:
def lasso_cdg(bstart,data,lmbda,epsilon=1e-06,maxiter=1000,standardized=False):
    p_coeff = bstart.size
    y = data[:,0]
    X = data[:,np.arange(1,data[0,:].size)]
    N,p = X.shape
    if p!=p_coeff:
        print("Error: Covariate number is not compatible with given coefficient vector")
        return
    else:
        if standardized==False:
            X = X / (np.linalg.norm(X,axis = 0))
        keyDict = {"estimate","objectives","steps","status"}
        output = dict([(key, []) for key in keyDict])
        for i in range(maxiter):
            for j in range(p):
                obj = lambda x : lasso_objective(np.hstack((bstart[0:j],x,bstart[j+1:p])),data,lmbda)
                res = minimize(obj, x0=bstart[j], method='Nelder-Mead')
                output["estimate"] = np.hstack((bstart[0:j],res.x,bstart[j+1:p]))
                output["objectives"] = res.fun
                output["steps"] = np.linalg.norm(bstart-output["estimate"],ord=np.inf)
                if output["steps"]<epsilon:
                    output["status"] = "convergence"
                    return output                             
                bstart = output["estimate"]                            
        output["status"] = "maxiter exceed"        
        return output        

                
                
            